# **Winequality**

# **SVM: Máquina de Vector de Soporte / Máquinas de soporte vectorial**


## **Importar Librerías**

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# MODULO DE MVS (SVC: Support Vector Classification)
from sklearn.svm import SVC

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

## **Importar datos**

In [11]:
datos = pd.read_csv("./winequality.csv", sep=',')
datos.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,12.2,0.450,0.49,1.4,0.075,3.0,6.0,0.99690,3.13,0.63,10.4,5
1,12.2,0.450,0.49,1.4,0.075,3.0,6.0,0.99690,3.13,0.63,10.4,5
2,8.6,0.315,0.40,2.2,0.079,3.0,6.0,0.99512,3.27,0.67,11.9,6
3,9.8,0.340,0.39,1.4,0.066,3.0,7.0,0.99470,3.19,0.55,11.4,7
4,8.0,0.600,0.08,2.6,0.056,3.0,7.0,0.99286,3.22,0.37,13.0,5


## **Distribución de clases**

In [12]:
print(datos.groupby('quality').size())

quality
4     63
5    681
6    633
7    216
dtype: int64


## **Separación de datos y normalización**

In [13]:
# separar variables predictoras y etiqueta
X = datos[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol']].values
y = datos['quality'].values

# Separa datos en entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Normalizar
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


## **Entrenamiento del modelo**

```

Definición de modelo con un kernel radial (rbf)
El parámetro "C" (parámetro de penalización) controla el trade-off entre el ajuste a los datos de entrenamiento y la complejidad del modelo.
- Un valor más alto de C indica que se da más importancia a la clasificación correcta de los datos de entrenamiento (ajuste preciso) lo que puede llevar a una frontera de decisión más compleja.
- Un valor más bajo de C da más importancia a lograr una mayor separación de las clases (frontera de decisión más simple).
En este caso, C se establecerá en 1, lo que indica un enfoque de equilibrio entre el ajuste y la simplicidad del modelo.

```

In [61]:
# definición del modelo
modelo = SVC(kernel='rbf', C=9)

# entrenamiento
modelo.fit(X_train, y_train)

print('Exactitud de SVM en la clasificacion de datos de entrenamiento: {:.2f}'.format(modelo.score(X_train, y_train)))

print('Exactitud de SVM en la clasificacion de datos de tes: {:.2f}'.format(modelo.score(X_test, y_test)))


Exactitud de SVM en la clasificacion de datos de entrenamiento: 0.72
Exactitud de SVM en la clasificacion de datos de tes: 0.63


## **Matriz de confusión y exactitud**

In [58]:
from sklearn.metrics import confusion_matrix
pred = modelo.predict(X_test)
matriz = confusion_matrix(y_test, pred)
print(matriz)

[[  5   9   4   0]
 [  8 111  38   2]
 [  5  56  86  15]
 [  0  10  15  35]]


In [59]:
# accuracy del modelo
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, pred)
print(accuracy)

0.5939849624060151


## **Predicción**

In [65]:
# predicción del modelo
y_pred = modelo.predict([[2.21, 0.458, 0.49, 1.5, 0.075, 4.0, 6.0, 0.99690, 3.13, 0.63, 10.4]])
print(y_pred)

[5]


In [9]:
# Adaptaciones de Dr. César Beltrán
C = [1, 2, 3, 4, 5, 10]
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
gamma = [0.001, 0.01, 0.1, 0.5, 0.9]

# 6 x 4 x 5 cant. experimentos
cuadricula_busqueda = {'C': C,
                       'kernel': kernel,
                       'gamma': gamma}

# Busqueda aleatoria en cuadricula con validación cruzada
from sklearn.model_selection import RandomizedSearchCV

modelo = SVC()

modelo_resultado = RandomizedSearchCV(estimator=modelo, param_distributions=cuadricula_busqueda, n_iter=50, cv=3, verbose=2, random_state=42)

modelo_resultado.fit(X_train, y_train)

modelo_resultado.score(X_train, y_train)


Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] END .....................C=3, gamma=0.01, kernel=linear; total time=   5.2s
[CV] END .....................C=3, gamma=0.01, kernel=linear; total time=   6.9s
[CV] END .....................C=3, gamma=0.01, kernel=linear; total time=   8.1s
[CV] END ....................C=3, gamma=0.01, kernel=sigmoid; total time=   0.1s
[CV] END ....................C=3, gamma=0.01, kernel=sigmoid; total time=   0.0s
[CV] END ....................C=3, gamma=0.01, kernel=sigmoid; total time=   0.1s
[CV] END .....................C=1, gamma=0.01, kernel=linear; total time=   2.0s
[CV] END .....................C=1, gamma=0.01, kernel=linear; total time=   0.9s
[CV] END .....................C=1, gamma=0.01, kernel=linear; total time=   3.2s
[CV] END .....................C=3, gamma=0.5, kernel=sigmoid; total time=   0.0s
[CV] END .....................C=3, gamma=0.5, kernel=sigmoid; total time=   0.0s
[CV] END .....................C=3, gamma=0.5, k

# Mejor modelo

In [13]:
modelo_resultado.best_estimator_

SVC(C=4, gamma=0.5, kernel='poly')